In [1]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
import requests, os
import pandas as pd

In [3]:
# You need to add the infos in the .env file
load_dotenv()

ES_HOST = os.getenv("ES_HOST")
ES_PASSWORD = os.getenv("ES_PASSWORD")
ES_USER = os.getenv("ES_USER")
es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))
INDEX_PROJECTS = f'scanr-projects'

from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))

In [26]:
user_query = 'spintronic' # TO CHANGE!

In [17]:
payload = {
    'size': 10000,
    '_source': # the fields we want to get 
    ['label.*',
  'publications',
  'participants.label',
  'participants.structure.id',
  'participants.structure.label.*',
  'participants.structure.mainAddress.*',
  'year',
  'type',
  'id',
  'keywords.*'],
 'sort': [{'_score': {'order': 'desc'}}, {'id.keyword': {'order': 'desc'}}],
 'highlight': {'number_of_fragments': 3,
  'fragment_size': 125,
  'pre_tags': ['<strong>'],
  'post_tags': ['</strong>'],
  'fields': {'label.default': {},
   'description.default': {},
   'domains.label.default': {}}},
 'query': {'bool': {'must': [{'query_string': {'query': user_query,
      'fields':  # the fields to search in
    ['id^3',
       'label.*^3',
       'title_abs_text^3',
       'acronym.*^3',
       'description.*^2',
       'domains.label.*',
       'call.id',
       'call.label',
       'action.id',
       'action.label.*',
       'keywords.*',
       'publications.id',
       'publications.title.*',
       'publications.authors.fullName',
       'publications.authors.person',
       'publications.domains.label.*',
       'publications.domains.code',
       'participants.structure.label.*',
       'participants.structure.acronym.*',
       'participants.structure.id',
       'participants.label.*']}}],
   'filter': []}}}

In [18]:
res = es.search(index=INDEX_PROJECTS, request_cache=False, request_timeout=6000, body=payload)

In [19]:
data = [e['_source'] for e in res['hits']['hits']]

In [20]:
df_results = pd.DataFrame(data)

In [21]:
len(df_results)

841

In [23]:
df_results.type.value_counts()

ANR                          337
H2020                        206
FP7                          145
Horizon Europe               126
PIA                           23
Partenariat Hubert Curien      2
iLab                           1
Thèses CIFRE                   1
Name: type, dtype: int64

In [24]:
df_results

,id,type,year,participants,label,keywords,publications
0,705222,H2020,2015,[{'label': {'default': 'Aalto University__-__7...,{'en': 'Controlling interfacial spin effects i...,"{'en': ['spin-orbit coupling', 'electrical con...",[]
1,709151,H2020,2015,[{'label': {'default': 'Johannes Gutenberg Uni...,{'en': 'The research of spin orbit torques in ...,"{'en': ['spintronics', 'domain walls', 'spin o...",[]
2,101108730,Horizon Europe,2022,"[{'structure': {'id': 'grid.424584.b', 'mainAd...",{'en': 'Spin-obit torque heterostructures base...,"{'en': ['spin orbite torque', 'graphene', 'spi...",[]
3,955671,H2020,2020,"[{'structure': {'id': '775685019', 'mainAddres...","{'en': 'Spin-orbit materials, emergent phenome...","{'en': ['spin orbitronics', 'spin-orbit torque...","[{'id': 'doi10.1038/s41467-023-37866-2', 'proj..."
4,899896,H2020,2019,[{'label': {'default': 'Institut Català de Nan...,{'en': 'Topological Insulator-Based Spin Orbit...,"{'en': ['Non-volatile memories', ' spintronics...",[]
...,...,...,...,...,...,...,...
836,ANR-16-IDEX-0008,PIA,2016,[{'label': {'default': 'Ecole Nationale Supéri...,{'fr': 'PSI'},"{'fr': ['Campus International, Innovation et t...","[{'id': 'doi10.1051/e3sconf/202017006006', 'pr..."
837,ANR-16-IDEX-0003,PIA,2016,"[{'structure': {'id': '200212702C', 'mainAddre...",{'fr': 'FUTURE'},"{'fr': ['Ville', 'Interdisciplinarité', 'Multi...","[{'id': 'doi10.1002/admi.202400827', 'projects..."
838,ANR-10-IDEX-0002,PIA,2010,[{'label': {'default': 'INSERM Paris 13__-__AN...,"{'fr': 'Par-delà les frontières, l'Université ...","{'fr': ['Excellence, Attractiveness, Multidisc...",[{'id': 'doi10.1103/physrevmaterials.8.054802'...
839,ANR-11-LABX-0014,PIA,2011,"[{'structure': {'id': '200812288B', 'mainAddre...",{'fr': 'Réseau national sur GaN'},"{'fr': ['GaN', 'nitrure de gallium', 'optoélec...","[{'id': 'doi10.1088/1361-6528/ab94e1', 'projec..."
